In [20]:
# pip нужно только один раз внутри venv/conda
# !pip install -U transformers peft pandas openpyxl tqdm sentencepiece

import torch, pandas as pd, numpy as np, tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from pathlib import Path

# ───── файлы ────────────────────────────────────────────────
BASE_ID       = "sberbank-ai/rugpt3small_based_on_gpt2"
ADAPTER_DIR   = Path("rugpt3_lora_adapter")          # где вы сохранили LoRA
SRC_XLSX      = Path("unique_texts_200_formatted_golden_standard.xlsx")
DST_XLSX      = Path("unique_texts_200_formatted_golden_standard_rugpt_v2.xlsx")

In [22]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

tok = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)
tok.pad_token = tok.eos_token          # GPT-семейство не знает PAD

base = AutoModelForCausalLM.from_pretrained(
    BASE_ID,
    torch_dtype=torch.float16,         # MPS работает быстрее на fp16
).to(device)

model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()
print("✅ модель и LoRA на", device)

✅ модель и LoRA на mps


In [24]:
PROMPT = "Упрости текст, сохраняя смысл:\n\n{src}\n\nУпрощённая версия:"

@torch.inference_mode()
def simplify(text, temp=0.7, top_p=0.9, max_new=512):
    prompt = PROMPT.format(src=text)
    ids = tok(prompt, return_tensors="pt").to(device)
    gen = model.generate(
        **ids,
        do_sample=True,
        temperature=temp,           # ↑ температура → меньше копипаста
        top_p=top_p,
        repetition_penalty=1.2,     # штраф за повтор слова
        no_repeat_ngram_size=4,     # запрет 4-грамм
        max_new_tokens=max_new,
        eos_token_id=tok.eos_token_id,
    )
    return tok.decode(gen[0], skip_special_tokens=True)\
             .split("Упрощённая версия:")[-1].strip()

In [26]:
# читаем исходный файл
df = pd.read_excel(SRC_XLSX)

# создаём столбец, если его нет
out_col = "finetuned_model_rugpt"
if out_col not in df.columns:
    df[out_col] = np.nan

# цикл с автосохранением после каждой строки
for i in tqdm.tqdm(range(len(df)), desc="Генерирую"):
    if pd.isna(df.at[i, out_col]):
        try:
            src_text = str(df.iloc[i, 0])            # первый столбец
            df.at[i, out_col] = simplify(src_text)
        except Exception as e:
            df.at[i, out_col] = f"[ERROR: {e}]"
    df.to_excel(DST_XLSX, index=False)               # сохраняем на каждом шаге

print("🎉  Всё готово →", DST_XLSX)


Генерирую:   0%|                                        | 0/171 [00:00<?, ?it/s]/var/folders/0v/tzcygl612vvdz00kp8wyv3th0000gn/T/ipykernel_48238/1220501331.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '## Как настроить подключение к Интернету через панель настроек  
   - Вводите код активации **Запуск панели** или **Вход"**;   - Во всплывающем списке выбираете пункт **Свойство "Паркнет".** Затем нажмите кнопку **«Очистить компьютер»** – откроется окошко с правами администратора доступа. Вход открывается только после того как вы активируете приложение «Паркет». Теперь система будет работать без сбоев и зависаний.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, out_col] = simplify(src_text)
Генерирую: 100%|██████████████████████████████| 171/171 [12:11<00:00,  4.28s/it]

🎉  Всё готово → unique_texts_200_formatted_golden_standard_rugpt_v2.xlsx


In [1]:
import torch, pandas as pd, numpy as np, tqdm, time
from transformers import AutoTokenizer, AutoModelForCausalLM
from pathlib import Path

BASE_ID       = "sberbank-ai/rugpt3small_based_on_gpt2"
SRC_XLSX      = Path("unique_texts_200_formatted_golden_standard.xlsx")
DST_XLSX      = Path("unique_texts_200_formatted_golden_standard_rugpt_base.xlsx")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

tok = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)
tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_ID,
    torch_dtype=torch.float16,
).to(device)

model.eval()
print("✅ базовая модель загружена на", device)

# Простой прямой промпт
PROMPT = """Переписать следующий текст более простыми словами:

"{text}"

Упрощенный текст:"""

@torch.inference_mode()
def simplify(text, max_new=256, pause=3.0):
    # Пауза перед генерацией
    time.sleep(pause)
    
    prompt = PROMPT.format(text=text)
    ids = tok(prompt, return_tensors="pt").to(device)
    
    gen = model.generate(
        **ids,
        do_sample=True,
        temperature=0.6,
        max_new_tokens=max_new,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.eos_token_id,
    )
    
    output = tok.decode(gen[0], skip_special_tokens=True)
    
    # Простое извлечение результата после маркера
    if "Упрощенный текст:" in output:
        return output.split("Упрощенный текст:")[-1].strip()
    else:
        # Если маркер не найден
        return output.split(text)[-1].strip()

# Чтение исходного файла
df = pd.read_excel(SRC_XLSX)

# Создание столбца, если его нет
out_col = "base_model_rugpt"
if out_col not in df.columns:
    df[out_col] = np.nan

# Преобразуем столбец в строковый тип
df[out_col] = df[out_col].astype('object')

# Цикл с автосохранением после каждой строки
for i in tqdm.tqdm(range(len(df)), desc="Генерирую базовой моделью"):
    if pd.isna(df.at[i, out_col]):
        try:
            src_text = str(df.iloc[i, 0])
            print(f"\nОбрабатываю текст {i+1}/{len(df)}")
            
            # Длительная пауза между обработками разных текстов
            if i > 0:
                time.sleep(7.0)
                
            result = simplify(src_text, pause=2.0)
            df.at[i, out_col] = result
        except Exception as e:
            print(f"Ошибка при обработке строки {i}: {e}")
            df.at[i, out_col] = f"[ERROR: {e}]"
        
        # Сохраняем после каждой обработанной строки
        df.to_excel(DST_XLSX, index=False)

print("🎉  Всё готово →", DST_XLSX)

✅ базовая модель загружена на mps


Генерирую базовой моделью:   0%|                        | 0/171 [00:00<?, ?it/s]


Обрабатываю текст 1/171


Генерирую базовой моделью:   1%|                | 1/171 [00:08<24:08,  8.52s/it]


Обрабатываю текст 2/171


...


Обрабатываю текст 170/171


Генерирую базовой моделью:  99%|█████████████▉| 170/171 [37:09<00:13, 13.00s/it]


Обрабатываю текст 171/171


Генерирую базовой моделью: 100%|██████████████| 171/171 [37:21<00:00, 13.11s/it]

🎉  Всё готово → unique_texts_200_formatted_golden_standard_rugpt_base.xlsx
